In [ ]:
# importing packages for scraping an data wrangling

import pandas as pd
import numpy as np
import time
import selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


chromeOptions = Options()
chrome_options = Options()

In [ ]:
#first Layer Scraping - clinical trial identifier, title, and status - using selenium and google chromedriver

driver = webdriver.Chrome("/Users/layalhammad/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/119/chromedriver", chrome_options = chromeOptions)

x=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

NCT = []
Status = []
Title = []

for i in x:    
#     Open URL
    driver.get(
    f'https://clinicaltrials.gov/search?cond=Cancer&term=liposomes&limit=100&page={i}')


    WebDriverWait(driver, 60)


    

    multiple_NCT = driver.find_elements(By.XPATH, "//div[contains(@class,'nct-id')]")


    multiple_Status = driver.find_elements(By.XPATH,"//div[contains(@class,'recruiting-status margin-bottom-1')]")                                  


    multiple_Title = driver.find_elements(By.XPATH,"//div[contains(@class,'card-container')]")




    for i, x, y, z in zip(multiple_NCT, multiple_Status, multiple_Title, multiple_Condition):
        NCT.append(i.text)
        Status.append(x.text)
        Title.append(y.text)
        Condition.append(z.text)
    
    WebDriverWait(driver, 60)
        

URL = list(map(lambda x: 'https://clinicaltrials.gov/study/'+ x,NCT))    
    
df = pd.DataFrame(list(zip(NCT,  Title, URL, Status, Condition)),
               columns =['NCT Number', 'Title', 'URL', 'Status', 'Condition subtype 1' ])


driver.close()

  

In [ ]:
#Second Layer Scraping - Part A - Study Details Tab 


driver = webdriver.Chrome("/Users/layalhammad/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/119/chromedriver", chrome_options = chromeOptions)

nan_nct =[] # in case of timeout exception
phase  = [] # Phase of clinical trial
gender = [] # gender of eligible participants
numend = [] # number of participants at the end
description = [] # description of clinical trial
publink = [] # publications at the study tab
start = [] # the beginning of the clinical trial
PEnd = [] # the primary end of the clinical trial
AEnd = [] # the actual end of the clinical trial
Condition = [] # condition subtypes of  the clinical trial



for i in NCT:    
# Open URL
    driver.get(
        f'https://clinicaltrials.gov/study/{i}')

# wait time until the page loads
    try:
        WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.XPATH, "//*[@class = 'tab-labels']")))

    except TimeoutException:
        nan_nct.append(i)
        pass    


# scraping text of variables

    try:
        multiple_start = driver.find_element(By.XPATH, "//*[@class= 'study-overview-item-title' and contains(text(), 'Start')]/following-sibling::*")
        start.append(multiple_start.text)

    except NoSuchElementException:
        start.append("NAN")
        pass


    try:
        multiple_PEnd = driver.find_element(By.XPATH, "//*[@class= 'study-overview-item-title' and contains(text(), 'Primary Completion')]/following-sibling::*")
        PEnd.append(multiple_PEnd.text)

    except NoSuchElementException:
        PEnd.append("NAN")
        pass


    try:
        multiple_AEnd = driver.find_element(By.XPATH, "//*[@class= 'study-overview-item-title' and contains(text(), 'Study Completion')]/following-sibling::*")
        AEnd.append(multiple_AEnd.text)

    except NoSuchElementException:
        AEnd.append("NAN")
        pass


    try:
        multiple_phase = driver.find_element(By.XPATH, "//*[@id='study-overview']/ctg-study-overview/div/div/div[2]/div/div[4]/ctg-enum-value/span")
        phase.append(multiple_phase.text)

    except NoSuchElementException:
        phase.append("NAN")
        pass


    try:
        multiple_gender = driver.find_element(By.XPATH, "//*[@id='participation-criteria']/div[2]/div/div[2]/div[2]/div[4]/ctg-enum-value/span")
        gender.append(multiple_gender.text)

    except NoSuchElementException:
        gender.append("NAN")
        pass


    try:  
        multiple_numend = driver.find_element(By.XPATH, "//*[@class= 'study-overview-item-title' and contains(text(), 'Enrollment ')]/following-sibling::*")
        numend.append(multiple_numend.text)

    except NoSuchElementException:
        numend.append("NAN")
        pass


    try:
        multiple_description = driver.find_element(By.XPATH, "//*[@id='detailed-description']")
        description.append(multiple_description.text)

    except NoSuchElementException:
        description.append("NAN")
        pass
    
    
    try:
        multiple_condition = driver.find_elements(By.XPATH, "//div[@class= 'conditions-item-text-pad-bottom']")
        Condition.append([e.text for e in multiple_condition])

    except NoSuchElementException:
        Condition.append("NAN")
        pass

# scraping publication links of clinical trials   
    try:
        multiple_pubres = driver.find_element(By.XPATH, "//*[@id = 'publications']").find_elements(By.TAG_NAME, 'a')
        multiple_pubres2 = [e.get_attribute("href") for e in multiple_pubres if e.get_attribute("href") is not None]
        publink.append(multiple_pubres2)

    except NoSuchElementException: 
        publink.append("NAN")
        pass    
    

driver.close()


In [ ]:
# create a dataframe of variables

dffirst = pd.DataFrame(list(zip(NCT,  phase, gender, publink, numend, description, start , PEnd, AEnd)),
                columns =['NCT Number', 'Phases', 'Gender', 'Publications Link', 'Num  participants at end','Description', 'Start Date', 'Primary End', 'Actual End' ])


In [ ]:
# Save the dataframe as CSV
dffirst.to_csv('LiposomesA.csv')

In [ ]:
#Second Layer scraping - Part B - Table Tab

driver = webdriver.Chrome("/Users/layalhammad/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/119/chromedriver", chrome_options = chromeOptions)



design = [] # design of clinical trial - Allocation, assignment model, and primary purpose
interv = [] # interventions used in clinical trial - main intervention, control, and dosage
arms = [] # study arms of the clinical trial - main intervention, control, and dosage
numor = [] # original number of participants
outcome = [] # description of outcome measure to be evaluated at the clinical trial
tabs = [] # an extra attribute to scrape for filtering posted and unposted results


for i in NCT:    
# Open URL
    driver.get(
        f'https://clinicaltrials.gov/study/{i}?tab=table')

# wait time until the page loads
    try:
        WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.XPATH, "//*[@class = 'tab-labels']")))
        
    except TimeoutException:
        nan_nct.append(i)
        pass    


# scraping text of variables

    try:
        multiple_design = driver.find_element(By.XPATH, "//*[@class= 'cell title-cell' and contains(div/span, 'Study Design')]/following-sibling::*")
        design.append(multiple_design.text)

    except NoSuchElementException:
        design.append("NAN")
        pass   

    
    
    try:
        multiple_interv = driver.find_element(By.XPATH, "//*[@class= 'cell title-cell' and contains(div/span, 'Intervention')]/following-sibling::*")
        interv.append(multiple_interv.text)

    except NoSuchElementException:
        interv.append("NAN")
        pass   



    try:
        multiple_arms = driver.find_element(By.XPATH, "//*[@class= 'cell title-cell' and contains(div/span, 'Study Arms')]/following-sibling::*")
        arms.append(multiple_arms.text)

    except NoSuchElementException:
        arms.append("NAN")
        pass    



    try:
        multiple_numor = driver.find_element(By.XPATH, "//*[@class= 'cell title-cell' and contains(div/span, 'Original Enrollment')]/following-sibling::*")
        numor.append(multiple_numor.text)

    except NoSuchElementException:
        numor.append("NAN")
        pass



    try:
        multiple_outcome = driver.find_element(By.XPATH, "//*[@id='outcome-measures']")   # to find outcome measures titles
        outcome.append(multiple_outcome.text)

    except NoSuchElementException:
        outcome.append("NAN")
        pass

       
    try:
        multiple_tabs = driver.find_element(By.XPATH, "//*[@class = 'tab-labels']")
        tabs.append(multiple_tabs.text)

    except NoSuchElementException:
        tabs.append("NAN")
        pass
    

driver.close()





In [ ]:
dfsecond = pd.DataFrame(list(zip(NCT,  design ,interv, arms, numor, outcome ,tabs)),
        columns =['NCT Number', 'design', 'intervention table view','arms', 'Num randomized participants','outcome', 'tabs' ])

In [ ]:
# reading the filtered - posted - completed and terminated results to scrape the results tab
dfnan = pd.read_excel('Nan-Liposome-NCT.xlsx')
NCT = list(dfnan['NCT for D'])
nctc = [x for x in NCT if not(pd.isnull(x)) == True]

In [ ]:
#Second Layer Scraping - Part C - Results Tab


driver = webdriver.Chrome("/Users/layalhammad/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/119/chromedriver", chrome_options = chromeOptions)

intervention = [] # interventions used in clinical trial - main intervention, control, and dosage
meanage = [] # mean age of participants
male = [] # percentage of male participants
time = [] # timeframe of adverse effects
 pub = [] # results publications
orr = [] # overall response rate
pfs = [] ########## two variations progression-free and progression free ###########
bor = [] #best overall response - did not work on reference
pfss = [] #second version of progression



for i in nctc:    
#   # Open URL
    driver.get(
         f'https://clinicaltrials.gov/study/{i}?tab=results')


    
# wait time until the page loads    
    try:
        WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[@class = 'tab-labels']")))
        
    except TimeoutException:
        nan_nct.append(i)
        pass    
    
    
# scraping text of variables

    try:
        multiple_meanage = driver.find_element(By.XPATH, "//*[@class= 'sticky-group' and contains(.//span, 'Age, Continuous')]/following-sibling::*")
        meanage.append(multiple_meanage.text)

    except NoSuchElementException:
        meanage.append("NAN")
        pass


    try:
        multiple_male = driver.find_elements(By.XPATH, "//*[@class= 'sticky-group' and contains(.//span, 'Sex: Female, Male')]/following-sibling::tbody//*[contains(.//span, 'Male')]/following-sibling::td/*[@class='cell-percentage']")
        male.append([e.text for e in multiple_male])

    except NoSuchElementException:
        male.append("NAN")
        pass


    try:        
        multiple_Int = driver.find_element(By.XPATH, "//*[@class= 'module-details-title margin-top-1' and contains(text(),'Intervention/Treatment')]/following-sibling::*")
        intervention.append(multiple_Int.text)

    except NoSuchElementException:
        intervention.append("NAN")
        pass



    try:   
        multiple_time = driver.find_element(By.XPATH, "//*[@class= 'module-details-title' and contains(text(), 'Time Frame')]/following-sibling::*")
        time.append(multiple_time.text)

    except NoSuchElementException:
        time.append("NAN")
        pass
  
    
    
    
    try:
        multiple_orr = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Overall Response Rate')]/following-sibling::div[3]/child::*")
        orr.append(multiple_orr.text)

    except NoSuchElementException:
        orr.append("NAN")
        pass



    try:
        multiple_pfs = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Progression Free Survival')]/following-sibling::div[3]/child::*")
        pfs.append(multiple_pfs.text)

    except NoSuchElementException:
        pfs.append("NAN")
        pass    


    try:
        multiple_bor = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Best Overall Response')]/following-sibling::div[3]/child::*")
        bor.append(multiple_bor.text)

    except NoSuchElementException:
        bor.append("NAN")
        pass    

    
    try:
        multiple_pfss = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Progression-free Survival')]/following-sibling::div[3]/child::*")
        pfss.append(multiple_pfss.text)

    except NoSuchElementException:
        pfss.append("NAN")
        pass    
     
        
    
# scraping results publication links    
    try:
        multiple_pubres = driver.find_element(By.XPATH, "//*[@id = 'publications']").find_elements(By.TAG_NAME, 'a')
        multiple_pubres2 = [e.get_attribute("href") for e in multiple_pubres if e.get_attribute("href") is not None]
        pub.append(multiple_pubres2)

    except NoSuchElementException: 
        pub.append("NAN")
        pass    
    
driver.close()




In [ ]:
# running code for all clinical trials and creating a dataframe of all publications available
dfpubrest = pd.DataFrame(list(zip(NCT,pub )),columns =['NCT Number', 'Results Publications' ])
dfpubrest.to_csv('liposomespubrest.csv')

In [ ]:
# running code only for posted results and creating a dataframe
dfthird = pd.DataFrame(list(zip(nctc,  intervention, meanage, male, time,  orr, pfs, bor, pfss )),
columns =['NCT Number', 'intervention', 'meanage', 'Male', 'TimeFrame',
          'Overall Response Rate', 'Progression Free Survival', 'Best Overall Response', 'Progression-free'])
dfthird.to_csv('liposomesincompleteC.csv')

In [ ]:
#Second Layer Scraping - Part E - Results Tab / Adverse Effects and pharmacokinetics

driver = webdriver.Chrome("/Users/layalhammad/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/119/chromedriver", chrome_options = chromeOptions)

sae = [] # serious adverse events
oae = [] # other adverse events


auc = [] # Area under the Curve 0-infinity
cmax = [] # Cmax
tmax = [] # Tmax


for i in nctc:
# open url
    driver.get(
        f'https://clinicaltrials.gov/study/{i}?tab=results')

    
# wait time until the page loads  
    try:
        WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[@class = 'tab-labels']")))
        
    except TimeoutException:
        nan_nct.append(i)
        pass    


# scraping text of variables

    try:
        multiple_sae = driver.find_element(By.XPATH, "//tr[@class = 'sticky-group' and contains(.//span, 'Serious Adverse Events')]/following-sibling::tbody//*[contains(span, 'Total')]/following-sibling::*")
        sae.append(multiple_sae.text)

    except NoSuchElementException:
        sae.append("NAN")
        pass


    try:
        multiple_oae = driver.find_element(By.XPATH, "//tr[@class = 'sticky-group' and contains(.//span, 'Other (Not Including Serious) Adverse Events')]/following-sibling::tbody//*[contains(span, 'Frequency Threshold for Reporting Other Adverse Events')]")
        oae.append(multiple_oae.text)

    except NoSuchElementException:
        oae.append("NAN")
        pass
    
    
    try:
        multiple_auc = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Area Under Curve (AUC)')]/following-sibling::div[3]/child::*")
        auc.append(multiple_auc.text)

    except NoSuchElementException:
        auc.append("NAN")
        pass    



    try:
        multiple_cmax = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Cmax')]/following-sibling::div[3]/child::*")
        cmax.append(multiple_cmax.text)

    except NoSuchElementException:
        cmax.append("NAN")
        pass    


    try:
        multiple_tmax = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Tmax')]/following-sibling::div[3]/child::*")
        tmax.append(multiple_tmax.text)

    except NoSuchElementException:
        tmax.append("NAN")
        pass    


driver.close()    

In [ ]:
# running code only for posted results and creating a dataframe
dfE = pd.DataFrame(list(zip(nctc,  sae, oae, auc, cmax, tmax)),
 columns =['NCT Number', 'Serious Adverse Events', 'Other Adverse Events', 'Area Under Curve', 'Cmax', 'Tmax'])
dfE.to_csv('liposomesE.csv')

In [ ]:
# Scraping Extra outcome measures

driver = webdriver.Chrome("/Users/layalhammad/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/119/chromedriver", chrome_options = chromeOptions)

nan_nct = []
timeresp = [] # time to response
overallsurvival = [] # overall survival
ttp = [] # time to progression
cbr = [] # clinical benefit rate


for i in nctc:    
#     # Open URL
    driver.get(
        f'https://clinicaltrials.gov/study/{i}?tab=results')


    try:
        WebDriverWait(driver, 15).until(
        EC.visibility_of_element_located((By.XPATH, "//*[@class = 'tab-labels']")))
        
    except TimeoutException:
        nan_nct.append(i)
        pass    



    try:
        multiple_timeresp = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Time to Response')]/following-sibling::div[3]/child::*")
        timeresp.append(multiple_timeresp.text)

    except NoSuchElementException:
        timeresp.append("NAN")
        pass



    try:
        multiple_ttp = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Time to Progression')]/following-sibling::div[3]/child::*")
        ttp.append(multiple_ttp.text)

    except NoSuchElementException:
        ttp.append("NAN")
        pass    


    try:
        multiple_cbr = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Clinical Benefit Rate')]/following-sibling::div[3]/child::*")
        cbr.append(multiple_cbr.text)

    except NoSuchElementException:
        cbr.append("NAN")
        pass    

    
    try:
        multiple_overallsurvival = driver.find_element(By.XPATH, "//a[@class = 'group-panel group-panel__expanded' and contains(.//span, 'Overall Survival')]/following-sibling::div[3]/child::*")
        overallsurvival.append(multiple_overallsurvival.text)

    except NoSuchElementException:
        overallsurvival.append("NAN")
        pass    
    

driver.close()


In [ ]:
# running code and creating a dataframe
dfDnan = pd.DataFrame(list(zip(nctc,  overallsurvival, timeresp, ttp, cbr)),
columns =['NCT Number', 'Overall Survival', 'Time to Response', 'Time to Progression', 'Clinical Benefit Rate'])
dfDnan.to_csv('LiposomesExtraoutcome.csv')